In [1]:
from nested_lookup import nested_lookup
import json
import math
from pdfminer.high_level import extract_text,extract_pages
from pdfminer.layout import LAParams, LTContainer, LTTextLine


In [2]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LAParams, LTContainer, LTTextLine

def extract_layout(pdf_path):
    # Create a layout parameters object
    laparams = LAParams()

    # Parse the layout information
    layout_objects = []
    page_height=0
    page_width=0
    for page_layout in extract_pages(pdf_path, laparams=laparams):
        for obj in page_layout:
            if isinstance(obj, LTContainer):
                for text_line in obj:
                    if isinstance(text_line, LTTextLine):
                        layout_objects.append(text_line)
  

    page_height,page_width=page_layout.height,page_layout.width
    return layout_objects,(page_height,page_width)

# Example usage
pdf_path = '../data/iron-man-2008.pdf'
layout_objects,page_dim = extract_layout(pdf_path)





In [4]:
with open("iron.txt","w") as f:
    for i in layout_objects:
        f.write(i.get_text())


In [350]:
page_dim

(792, 612)

## Scene Type

In [351]:
import re
def is_int_ext_line(line):
    line=line.strip()
    if line.startswith("INT.") or line.startswith("EXT."):
        return True
    else:
        return False
    

## Transition

In [352]:
import re

transitionEnum = [
    "FADE IN",
    "FADE OUT",
    "FADE TO",  # Add a comma here
    "FADE UP",
    "JUMP CUT",
    "MATCH CUT",
    "SMASH MATCH CUT",
    "MATCH DISSOLVE",
    "CUT",
    "DISSOLVE",
    "FLASH CUT",
    "FREEZE FRAME",
    "IRIS IN",
    "IRIS OUT",
    "WIPE TO",
    "CUT TO",
    "CONTINUED",
    "TO WHITE",
    "TO BLACK"
]

def checkTransition(line, transitionEnum=None):
    line = line.strip().replace("\n", "")

    transitions_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(transition) for transition in transitionEnum) + r')\b', re.IGNORECASE)
    return bool(re.search(transitions_pattern, line))


## Character

In [353]:
def all_words_uppercase_character(text=None,words_lenght=5,bbox=None,page_dim=None,range_pos=(30,42)):
    # Remove non-alphabetic characters and split the text into words
    text = text.strip().replace("\n", "").replace("\.n","").replace("\u2019D","")
    # Update the regular expression to allow for numbers in words
    words = re.findall(r'\b[\w\d]+\b', text)

    print(words)

    # Check if all words, including those with numbers, are in uppercase

    position=(bbox[0]/page_dim[1])*100
    if len(words)<words_lenght and range_pos[0]<=position<=range_pos[1] : 
        return all(word.upper() == word for word in words)
    else:
        return False

In [354]:
def all_words_uppercase(text=None,words_lenght=5):
    # Remove non-alphabetic characters and split the text into words
    text = text.strip().replace("\n", "").replace("\u2019D","")
    # Update the regular expression to allow for numbers in words
    words = re.findall(r'\b[\w\d]+\b', text)

    print(words)

    # Check if all words, including those with numbers, are in uppercase


    

    if len(words)<words_lenght:
        return all(word.upper() == word for word in words)
    else:
        return False



def movie_character_position(movie_dict=None,frequency_count={}):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="character":
            frequency_count[math.ceil(value['bbox'][0])]=frequency_count.get(value['bbox'][0],0)+1
    frequency_count = dict(sorted(frequency_count.items(), key=lambda item: item[1],reverse=True))
    return frequency_count
            

def moviecharacter_precision_increase(movie_dict=None,max_value=0):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="character":
            if math.floor(value['bbox'][0])<=max_value<=math.ceil(value['bbox'][0]):
                print(i,math.ceil(value['bbox'][0]))
                movie_dict[i]['metadata']="character"
            else:
                movie_dict[i]['metadata']="unknown"
    return movie_dict
            



## Actionlines

In [355]:

def actionline_position(movie_dict=None,frequency_count={}):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            frequency_count[math.ceil(value['bbox'][0])]=frequency_count.get(value['bbox'][0],0)+1
    frequency_count = dict(sorted(frequency_count.items(), key=lambda item: item[1],reverse=True))
    return frequency_count
            

def actionline_increase_precision(movie_dict=None,max_value=0):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            if math.floor(value['bbox'][0])<=max_value<=math.ceil(value['bbox'][0]):
                print(i,math.ceil(value['bbox'][0]))
                movie_dict[i]['metadata']="action_line"
            else:
                movie_dict[i]['metadata']="unknown"
    return movie_dict
            

## Diaglouge


In [356]:

def dialogue_position(movie_dict=None,frequency_count={}):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            frequency_count[math.ceil(value['bbox'][0])]=frequency_count.get(value['bbox'][0],0)+1
    frequency_count = dict(sorted(frequency_count.items(), key=lambda item: item[1],reverse=True))
    return frequency_count
            

def dialogue_increase_precision(movie_dict=None,max_value=0):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            if math.floor(value['bbox'][0])<=max_value<=math.ceil(value['bbox'][0]):
                print(i,math.ceil(value['bbox'][0]))
                movie_dict[i]['metadata']="dialogue"
            else:
                movie_dict[i]['metadata']="unknown"
    return movie_dict
            

## Paranthesis

In [357]:
def is_whole_text_in_parentheses(text):
    text=text.strip().replace("\n","")
    pattern = r'^\(.+\)$'
    return bool(re.match(pattern, text))

In [358]:
is_whole_text_in_parentheses("(V.O.)\n")

True

## next line

In [359]:
def replace_patterns(text):
    text=text.strip()
    pattern = r'\b\d+\.\n'
    return re.sub(pattern, '', text)


In [360]:
def all_characters_are_numbers(s):
    s=s.replace(".","")
    return all(char.isdigit() for char in s)

In [361]:
def replace_next_line_empty_string(text):
    text=text.split("number:")[1].strip().replace("\n","")
    text=replace_patterns(text).strip()
    print("replace:",text)

    if len(text)==0 or all_characters_are_numbers(text):

        return True
    else:
        return False


In [362]:
movie_dict={}

In [363]:
def create_and_save_json(movie_dict, filename="pdfminer_Interstallar.json"):
    with open(filename, "w") as file:
        json.dump(movie_dict, file)

In [364]:
movie_dict={}

In [365]:
# Print layout information
for i, text_line in enumerate(layout_objects):
    # print(f"Text Line {i + 1}:")
    # print(f"   - Text: {text_line.get_text()}")
    # print(f"   - Position: ({text_line.x0}, {text_line.y0})")
    # print(f"   - Size: ({text_line.width}, {text_line.height})")
    # print("-------------------------")
    bbox=(text_line.x0,text_line.y0,text_line.width,text_line.height)
    sentence_text=text_line.get_text()
    if is_int_ext_line(sentence_text):
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"scene_info","bbox":bbox}
    elif checkTransition(sentence_text,transitionEnum=transitionEnum) and all_words_uppercase(sentence_text):
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"scene_transition","bbox":bbox}
    elif is_whole_text_in_parentheses(sentence_text):
        print("whole_text_paranthesis",sentence_text)
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"paranthesis","bbox":bbox}
    elif all_words_uppercase_character(text=sentence_text,bbox=bbox,page_dim=page_dim):
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"character","bbox":bbox}
    else:
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"unknown","bbox":bbox}


    

['IRON', 'MAN']
['by']
['Matt', 'Holloway', 'Art', 'Marcum']
['and']
['Mark', 'Fergus', 'Hawk', 'Ostby']
['Based', 'on', 'the', 'Marvel', 'Comic']
['Revisions', 'by']
['Matt', 'Holloway', 'Art', 'Marcum']
['Mark', 'Fergus', 'Hawk', 'Ostby']
['John', 'August']
['Current', 'Revisions', 'by']
['Mark', 'Fergus', 'and', 'Hawk', 'Ostby']
['SALMON', '2']
['XX', 'XX', '07']
['05', '09', '07']
['BUFF', '2']
['GOLDENROD', '2', '05', '02', '07']
['05', '01', '07']
['GREEN', '2']
['04', '24', '07']
['YELLOW', '2']
['04', '13', '07']
['PINK', '2']
['04', '12', '07']
['BLUE', '2']
['04', '05', '07']
['WHITE', '2']
['03', '30', '07']
['TAN']
['03', '23', '07']
['CHERRY']
['03', '21', '07']
['SALMON']
['03', '15', '07']
['BUFF']
['03', '12', '07']
['GOLDENROD']
['03', '09', '07']
['GREEN']
['03', '08', '07']
['YELLOW']
['02', '28', '07']
['PINK']
['02', '20', '07']
['BLUE']
['01', '24', '07']
['WHITE']
['MARVEL', 'STUDIOS']
['ALL', 'RIGHTS', 'RESERVED', 'COPYRIGHT', '2007', 'MARVEL', 'STUDIOS', 'INC',

In [366]:
frequency_count=movie_character_position(movie_dict=movie_dict)
movie_dict=moviecharacter_precision_increase(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])

frequency_count=actionline_position(movie_dict=movie_dict)
movie_dict=actionline_increase_precision(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])

frequency_count=movie_character_position(movie_dict=movie_dict)
movie_dict=moviecharacter_precision_increase(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])

frequency_count=dialogue_position(movie_dict=movie_dict)
movie_dict=dialogue_increase_precision(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])

98:number:© 2007 MARVEL STUDIOS, INC.
 230
144:number:© 2007 MARVEL STUDIOS, INC.
 230
191:number:© 2007 MARVEL STUDIOS, INC.
 230
238:number:© 2007 MARVEL STUDIOS, INC.
 230
280:number:© 2007 MARVEL STUDIOS, INC.
 230
334:number:© 2007 MARVEL STUDIOS, INC.
 230
386:number:© 2007 MARVEL STUDIOS, INC.
 230
435:number:© 2007 MARVEL STUDIOS, INC.
 230
459:number:© 2007 MARVEL STUDIOS, INC.
 230
505:number:© 2007 MARVEL STUDIOS, INC.
 230
516:number:© 2007 MARVEL STUDIOS, INC.
 230
558:number:© 2007 MARVEL STUDIOS, INC.
 230
610:number:© 2007 MARVEL STUDIOS, INC.
 230
621:number:© 2007 MARVEL STUDIOS, INC.
 230
642:number:© 2007 MARVEL STUDIOS, INC.
 230
653:number:© 2007 MARVEL STUDIOS, INC.
 230
696:number:© 2007 MARVEL STUDIOS, INC.
 230
740:number:© 2007 MARVEL STUDIOS, INC.
 230
775:number:© 2007 MARVEL STUDIOS, INC.
 230
793:number:© 2007 MARVEL STUDIOS, INC.
 230
839:number:© 2007 MARVEL STUDIOS, INC.
 230
853:number:© 2007 MARVEL STUDIOS, INC.
 230
903:number:© 2007 MARVEL STUDIOS,

In [367]:
def page_number_tag(movie_dict=None):
    for i in movie_dict.keys():
        value=movie_dict[i]
        
        if value['metadata']=="unknown":
            if replace_next_line_empty_string(i):
                movie_dict[i]['metadata']="page_number"
                print(i)
    return movie_dict

In [368]:
movie_dict=page_number_tag(movie_dict=movie_dict)

replace: IRON MAN
replace: by
replace: Matt Holloway & Art Marcum
replace: and
replace: Mark Fergus & Hawk Ostby
replace: Based on the Marvel Comic
replace: Revisions by:
replace: Matt Holloway & Art Marcum
replace: Mark Fergus & Hawk Ostby
replace: John August
replace: Current Revisions by:
replace: Mark Fergus and Hawk Ostby
replace: SALMON #2:
replace: XX/XX/07
replace: 05/09/07
replace: BUFF #2:
replace: GOLDENROD #2: 05/02/07
replace: 05/01/07
replace: GREEN #2:
replace: 04/24/07
replace: YELLOW #2:
replace: 04/13/07
replace: PINK #2:
replace: 04/12/07
replace: BLUE #2:
replace: 04/05/07
replace: WHITE #2:
replace: 03/30/07
replace: TAN:
replace: 03/23/07
replace: CHERRY:
replace: 03/21/07
replace: SALMON:
replace: 03/15/07
replace: BUFF:
replace: 03/12/07
replace: GOLDENROD:
replace: 03/09/07
replace: GREEN:
replace: 03/08/07
replace: YELLOW:
replace: 02/28/07
replace: PINK:
replace: 02/20/07
replace: BLUE:
replace: 01/24/07
replace: WHITE:
replace: MARVEL STUDIOS
replace: ALL RI

replace: Obadiah’s face sets.  He begins climbing up the scaffolding
replace: to the Dynamo.
replace: B145
replace: B145
replace: We HEAR A DOOR SPLINTERING.  Rhodey rushes in and finds --
replace: Traces of blood on the floor.  The collapsed shelving unit.
replace: Tony staggering to stay on his feet.
replace: RHODEY
replace: TONY--
replace: TONY
replace: Where’s Pepper?
replace: RHODEY
replace: Don’t worry, she’s with the Feds.
replace: They’re on their way to Stark to
replace: arrest Obadiah.
replace: TONY
replace: They’re going to need a lot more
replace: than a few Agents.  Give me hand.
replace: SALMON #2
replace: XX/XX/07
replace: 96A.
replace: C145
replace: C145
replace: The door swings open and Tony and Rhodey enter.  Rhodey stops
replace: shocked, his eyes playing over --
replace: The Hall of Armor.  The hanging Mark II suit.  The weaponized
replace: battle-ready Mark III.  Components, rows of helmets, boots,
replace: gauntlets and future design prototypes littered everywhere

In [369]:
def charchater_dict(movie_dict=None):
    character_set={}
    for i in movie_dict.keys():
        value=movie_dict[i]
        raw_text=i.split("number:")[1].replace("\n","").strip().upper()
        if value['metadata']=="character":
            character_set[raw_text]="character"
    return character_set            
        

character_set=charchater_dict(movie_dict=movie_dict)

In [370]:
character_set

{'© 2007 MARVEL STUDIOS, INC.': 'character'}

In [371]:
"(cont\u2019d)"

'(cont’d)'

In [372]:
def monkey_patch(text,patch_words=[]):
    print("monkey_patch",text)
    for i in patch_words:
        if i in text:
            return True
    return False


In [373]:
def character_increase_precison(movie_dict=None,character_set=character_set,patch_words=["(CONT'D)","(CONT’D)"]):
    for i in movie_dict.keys():
        value=movie_dict[i]
        raw_text=i.split("number:")[1].replace("\n","").replace(".\n","").strip().upper()
        if value['metadata']=="unknown":
            if (raw_text in character_set.keys()) or monkey_patch(raw_text,patch_words=patch_words):
                movie_dict[i]['metadata']="character" 
    return movie_dict           

movie_dict=character_increase_precison(movie_dict=movie_dict,character_set=character_set)

monkey_patch IRON MAN
monkey_patch BY
monkey_patch MATT HOLLOWAY & ART MARCUM
monkey_patch AND
monkey_patch MARK FERGUS & HAWK OSTBY
monkey_patch BASED ON THE MARVEL COMIC
monkey_patch REVISIONS BY:
monkey_patch MATT HOLLOWAY & ART MARCUM
monkey_patch MARK FERGUS & HAWK OSTBY
monkey_patch JOHN AUGUST
monkey_patch CURRENT REVISIONS BY:
monkey_patch MARK FERGUS AND HAWK OSTBY
monkey_patch SALMON #2:
monkey_patch XX/XX/07
monkey_patch 05/09/07
monkey_patch BUFF #2:
monkey_patch GOLDENROD #2: 05/02/07
monkey_patch 05/01/07
monkey_patch GREEN #2:
monkey_patch 04/24/07
monkey_patch YELLOW #2:
monkey_patch 04/13/07
monkey_patch PINK #2:
monkey_patch 04/12/07
monkey_patch BLUE #2:
monkey_patch 04/05/07
monkey_patch WHITE #2:
monkey_patch 03/30/07
monkey_patch TAN:
monkey_patch 03/23/07
monkey_patch CHERRY:
monkey_patch 03/21/07
monkey_patch SALMON:
monkey_patch 03/15/07
monkey_patch BUFF:
monkey_patch 03/12/07
monkey_patch GOLDENROD:
monkey_patch 03/09/07
monkey_patch GREEN:
monkey_patch 03/08

In [374]:
monkey_patch("(CONT’D)",patch_words=["(CONT'D)","(CONT’D)","(cont’d)"])

monkey_patch (CONT’D)


True

In [375]:
def introduction_tag(movie_dict=None):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata'] in ["scene_info",'action_line',"dialogue","character"]:
            break
        else:
            if value['metadata']=="unknown":
                movie_dict[i]['metadata']="intro"
    return movie_dict

    

In [376]:
movie_dict

{'1:number:IRON MAN\n': {'metadata': 'unknown',
  'bbox': (278.0, 651.408, 56.19720000000001, 12.0)},
 '2:number:by\n': {'metadata': 'unknown',
  'bbox': (299.0, 615.408, 14.199599999999975, 12.0)},
 '3:number:Matt Holloway & Art Marcum\n': {'metadata': 'unknown',
  'bbox': (215.0, 579.408, 182.18999999999988, 12.0)},
 '4:number:and\n': {'metadata': 'unknown',
  'bbox': (295.004, 567.408, 21.199199999999962, 12.0)},
 '5:number:Mark Fergus & Hawk Ostby\n': {'metadata': 'unknown',
  'bbox': (222.008, 555.408, 168.19079999999988, 12.0)},
 '6:number:Based on the Marvel Comic\n': {'metadata': 'unknown',
  'bbox': (218.012, 519.408, 175.1903999999999, 12.0)},
 '7:number:Revisions by:\n': {'metadata': 'unknown',
  'bbox': (260.012, 483.408, 91.19519999999994, 12.0)},
 '8:number:Matt Holloway & Art Marcum\n': {'metadata': 'unknown',
  'bbox': (215.012, 471.408, 182.18999999999988, 12.0)},
 '9:number:Mark Fergus & Hawk Ostby\n': {'metadata': 'unknown',
  'bbox': (222.008, 459.408, 168.190799999

In [377]:
movie_dict=introduction_tag(movie_dict=movie_dict)

In [378]:
create_and_save_json(movie_dict=movie_dict)

In [379]:
page_dim

(792, 612)

In [380]:
import re

# Example usage
result = all_words_uppercase("JiT 2\n")
print(result)  # Output: True


['JiT', '2']
False


In [381]:
replace_patterns("75.\n")

'75.'